In [39]:
import numpy as np
import pandas as pd
import os

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

from neuromaps.images import load_data, load_gifti, annot_to_gifti, relabel_gifti, construct_shape_gii
from neuromaps.datasets import fetch_annotation
from neuromaps.resampling import resample_images
from neuromaps.nulls import alexander_bloch, burt2020
from neuromaps.parcellate import Parcellater
from scipy.stats import pearsonr
from neuromaps import transforms 
from neuromaps.stats import compare_images
from neuromaps.nulls import hungarian

In [40]:
# define path
base_path = os.path.dirname(os.getcwd())

def load_atlas(base_path):
    """
    Load different parcellation files.
    """    
    atlas= {
        'dk_fsaverage_10k': (
            os.path.join(base_path, 'parcellations', 'atlas-desikankilliany_space-fsaverage_den-10k_hemi-L.label.gii.gz'),
            os.path.join(base_path, 'parcellations', 'atlas-desikankilliany_space-fsaverage_den-10k_hemi-R.label.gii.gz')
    ),
    'dk_fsaverage_164k': (
            os.path.join(base_path, 'parcellations', 'atlas-desikankilliany_space-fsaverage_den-164k_hemi-L.aparc-1.annot'),
            os.path.join(base_path, 'parcellations', 'atlas-desikankilliany_space-fsaverage_den-164k_hemi-R.aparc-1.annot')
    ),
    'dk_mni': os.path.join(base_path, 'parcellations', 'atlas-desikankilliany_space-MNI_res-1mm.nii.gz')
    }
    return (atlas) 

# create parcellaters 
def create_parcellaters(atlas):
    dk_fsaverage_10k = relabel_gifti(atlas['dk_fsaverage_10k'])
    parcellater_fs10k = Parcellater(dk_fsaverage_10k, 'fsaverage')
    
    dk_fsaverage_164k = annot_to_gifti(atlas['dk_fsaverage_164k'])  
    parcellater_fs164k = Parcellater(dk_fsaverage_164k, 'fsaverage')

    parcellater_mni = Parcellater(atlas['dk_mni'], 'MNI152')
    return (parcellater_fs10k, parcellater_fs164k, parcellater_mni) 

    
# load atlas 
atlas = load_atlas(base_path)
parcellater_fs10k, parcellater_fs164k, parcellater_mni = create_parcellaters(atlas)



In [41]:
# get turku & enigma data 

def parcellated_Turku(base_path):
    # get turku maps  
    img_L = load_data(os.path.join(base_path, 'data', 'lh.sig.nii'))
    img_R = load_data(os.path.join(base_path, 'data', 'rh.sig.nii'))
    turku_map = (construct_shape_gii(img_L), construct_shape_gii(img_R))

    turku_parc = parcellater_fs164k.fit_transform(turku_map, space='fsaverage', ignore_background_data=True)
    
    return turku_parc 
    
turku_parc = parcellated_Turku(base_path)
np.save(os.path.join(base_path, 'data' 'turku_parc.npy'), turku_parc)

# download enigma
enigma_file='ENIGMA_S32_partial_correlation_between_cortical_thickness_and_chlorpromazine_equivalents.csv' 
enigmamap = pd.read_csv(os.path.join(base_path, 'data',enigma_file))
enigmamap.drop([68, 69], inplace=True)  # Remove the last two rows
enigma_parc = enigmamap['partial_r'].to_numpy()


# download the regions for MNI152, take indecies of surface rois  
rois = pd.read_csv(os.path.join(base_path, 'parcellations' ,'atlas-desikankilliany.csv'))
rois = rois[(rois['structure'] == 'cortex')].index.to_numpy()


In [42]:
# get annotations 
annotations = list(fetch_annotation(source=['hcps1200',
                                            'raichle',
                                            'ding2010', 
                                            'finnema2016', 
                                            'dubois2015',
                                            'gallezot2010',
                                            'gallezot2017',
                                            'hillmer2016',
                                            'jaworska2020',
                                            'kaller2017',
                                            'kantonen2020',
                                            'laurikainen2018',
                                            'normandin2015',
                                            'radnakrishnan2018',
                                            'sandiego2015',
                                            'satterthwaite2014',
                                            'savli2012',
                                            'satterthwaite2014',
                                            'smith2017',
                                            'tuominen',
                                            'naganawa2020',
                                            'fazio2016']).keys())

annotations.extend(fetch_annotation(source=['norgaard2021', 'beliveau2017'], space='fsaverage').keys())
annotations.extend(fetch_annotation(source='margulies2016', desc='fcgradient01', return_single=False).keys())


In [43]:
# parcellate annotations

# initialize
parcellated = dict([])

# go over each annotation and parcellate depending on the space 
for (src, desc, space, den) in annotations:

    annot = fetch_annotation(source=src, desc=desc, space=space, den=den)
    
    if space == 'MNI152':
        parcellater = parcellater_mni
    elif space == 'fsaverage' and den == '164k':
        parcellater = parcellater_fs164k
    elif space == 'fsLR' and den == '164k':
        space = 'fsaverage'
        annot = transforms.fslr_to_fsaverage(annot, target_density='164k')
        parcellater = parcellater_fs164k
    elif space == 'fsLR' and den != '164k':
        # unfortunately for fsLR-4k we are upsampling to fsaverage-10k to parcellate but it should be fine
        space = 'fsaverage'
        annot = transforms.fslr_to_fsaverage(annot, target_density='10k')
        parcellater = parcellater_fs10k

    parcellated[desc] = parcellater.fit_transform(annot, space=space, ignore_background_data=True)

    # if subcortex included remove 
    if parcellated[desc].shape == (1,83):
        parcellated[desc] = parcellated[desc][0][rois]
  

/Users/laurituominen/anaconda3/lib/python3.10/site-packages/scipy/ndimage/_measurements.py:803: RuntimeWarning: invalid value encountered in divide
  return sum / numpy.asanyarray(count).astype(numpy.float64)


In [44]:
# get spins 
spins = pd.read_csv(os.path.join(base_path, 'parcellations', 
                                 'spins_hungarian_aparc+aseg_ctx.csv'), header=None)
nspins = spins.values.shape[1]


In [47]:
def calc_corrs(annotations, parc):
    """
    Calculate correlations & nulls between parcellated annotations & 
    parcellated antipsychotic effects on cortical thickness 
    """

    # initialize
    nulls = dict([])
    corrs = dict([])
    
    # go over annotations 
    for src, desc, space, den in annotations:
        if space == 'MNI152':
            parcellation=atlas['dk_mni']
            
        elif space == 'fsaverage' and den == '164k':
            parcellation=atlas['dk_fsaverage_164k']
            
        elif space == 'fsLR' and den == '164k':
            parcellation=atlas['dk_fsaverage_164k']
            
        elif space == 'fsLR' and den != '164k':
            parcellation=atlas['dk_fsaverage_10k']
        
        # empirical correlation between annotations and parc
        rho = pearsonr(parcellated[desc], parc)[0]
        
        # get 10k rotations 
        rotated = hungarian(data=parcellated[desc], n_perm=10000, spins=spins, parcellation=parcellation) 
        
        # get null 
        n = np.zeros((nspins, ))
        for i in range(nspins):
            n[i] = pearsonr(parc, rotated[:,i])[0]    
        
        # get p-value
        pspin = (1 + sum(abs(n) > abs(rho ))) / (nspins + 1)
    
        # store, multiply by -1 to make more intuitive, because smaller p-value/rho means bigger effect  
        corrs[src+'_'+desc] = ( (-1 * rho, pspin ) )
        nulls[src+'_'+desc] = n
        
    return(nulls, corrs)
    

In [48]:
nulls_turku, corrs_turku = calc_corrs(annotations, turku_parc)
nulls_enigma, corrs_enigma = calc_corrs(annotations, enigma_parc)


In [52]:
# save correlations & nulls 
np.savez(os.path.join(base_path, 'data', 'corrs_turku.npz'), **corrs_turku)
np.savez(os.path.join(base_path, 'data', 'nulls_turku.npz'), **nulls_turku)
np.savez(os.path.join(base_path, 'data', 'nulls_enigma.npz'), **nulls_enigma)
np.savez(os.path.join(base_path, 'data', 'corrs_enigma.npz'), **corrs_enigma)